In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from scipy import stats 
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import timeit
from datetime import datetime
from sklearn.model_selection import GridSearchCV
import os
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import make_scorer
from scipy.stats import shapiro 
from scipy.stats import norm, lognorm
import math
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import RobustScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

# Импорт Данных 

In [40]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
Id_answr_Array = df_test.Id.values
df = pd.concat([df_test.assign(indic="test"), df_train.assign(indic="train")])
df.sort_values(by=['Id'],inplace = True)
df.reset_index(inplace=True)
df.drop(['index'], axis=1,inplace = True)
df.drop(['Id'],axis=1,inplace = True)

pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 10000)
pd.options.display.max_rows = 100000

# Ответ

In [41]:
ans = pd.read_csv('Real Answers\Proper_Answers.csv')
ans_arr = ans.SalePrice.values

# Флаг для переключения RMSE и RMLSE при логарифмировании целевой переменной

In [42]:
flag = True

# LotFrontage

In [43]:
df['LotFrontage'] = df.groupby(['LotConfig'], sort=False)['LotFrontage'].apply(lambda x: x.fillna(x.mean()))

# Alley

In [44]:
df.Alley.fillna("NA", inplace = True)

# Работаем с BsmtQual, BsmtCond, BsmtFinType2, BsmtExposure и BsmtFinType1

In [45]:
cond1 = (0 < df.BsmtQual.isnull().astype(int) + df.BsmtCond.isnull().astype(int) + df.BsmtExposure.isnull().astype(int) \
+ df.BsmtFinType1.isnull().astype(int)  + df.BsmtFinType2.isnull().astype(int))  & \
(df.BsmtQual.isnull().astype(int) + df.BsmtCond.isnull().astype(int) + df.BsmtExposure.isnull().astype(int) \
+ df.BsmtFinType1.isnull().astype(int)  + df.BsmtFinType2.isnull().astype(int) < 5)
df.loc[332,'BsmtFinType2'] = 'Unf'
df.loc[2217,'BsmtQual'] = 'Fa'
df.loc[2218,'BsmtQual'] = 'Fa'
df.loc[948,'BsmtExposure'] = 'Mn'
df.loc[1487,'BsmtExposure'] = 'Av'
df.loc[2348,'BsmtExposure'] = 'Mn'
df.loc[2040,'BsmtCond'] = 'TA'
df.loc[2185,'BsmtCond'] = 'TA'
df.loc[2524,'BsmtCond'] = 'TA'
NA_Bath_condition = df.BsmtQual.isnull() & df.BsmtCond.isnull() & df.BsmtExposure.isnull() & df.BsmtFinType1.isnull() \
& df.BsmtFinType2.isnull() & (df.BsmtFinSF1 == 0) & (df.BsmtFinSF2 == 0) & (df.BsmtFinSF2 == 0) \
& (df.BsmtUnfSF == 0) & (df.TotalBsmtSF == 0) & (df.BsmtFullBath == 0) & (df.BsmtHalfBath == 0)
NA_Bath_columns_extended = ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','BsmtFinSF1','BsmtFinSF2','BsmtFinSF2',\
'BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath']
NA_Bath_columns = ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']
df[NA_Bath_columns] = df[NA_Bath_columns].fillna("NA")

# Работаем с переменными GarageType, GarageFinish, GarageCond GarageQual

In [46]:
mask1 = df.GarageType.isnull()
df.loc[mask1, ['GarageType','GarageCond','GarageQual','GarageFinish']] = df.loc[mask1, ['GarageType','GarageCond','GarageQual','GarageFinish']].fillna('NA')

# Работаем с переменной FireplaceQu 

In [47]:
df.FireplaceQu.fillna('NA', inplace = True)

# Работаем с переменной PoolQC 

In [48]:
mask = (df.PoolArea==0) & df.PoolQC.isna()
df.loc[mask,'PoolQC'] = df.loc[mask,'PoolQC'].fillna('NA')
df.loc[2420,'PoolQC'] = 'Ex'
df.loc[2503,'PoolQC'] = 'Ex'
df.loc[2599,'PoolQC'] = 'Fa'

# Разберемся с GarageYrBlt

In [49]:
mn=(df.GarageYrBlt - df.YearBuilt).mean()
df.GarageYrBlt = df.GarageYrBlt.fillna(df.YearBuilt + mn) # Лучше df1.GarageYrBlt.fillna(df1.YearBuilt + mn, inplace = True)

# Заполним на Fence и MiscFeature

In [50]:
df.Fence.fillna('NA',inplace = True)
df.MiscFeature.fillna('NA',inplace = True)

# Изучим MasVnrType и MasVnrArea

In [51]:
df.loc[2610,'MasVnrType'] = 'BrkCmn'
df.MasVnrArea.fillna(0, inplace = True)
df.MasVnrType.fillna('None', inplace = True)

# Доделаем оставшиеся NaN

In [52]:
arr = df.isnull().sum()[df.isnull().sum()>0].index[:len(df.isnull().sum()[df.isnull().sum()>0])-1]
df[arr] = df[arr].interpolate(method='pad') # Аналог fillna()
df.isnull().sum()[df.isnull().sum()>0]

SalePrice    1459
dtype: int64

# Преобразуем Object натуральными числами

In [53]:
obj_ind = df.dtypes[df.dtypes=='object'].index
obj_ind = obj_ind.delete(-1)
for ind in obj_ind :
    gr = df.groupby([ind]).mean().SalePrice.copy()
    gr.sort_values(inplace = True)
    for i in range(0,gr.shape[0]):
        gr[i] = int(i)
    dics = gr.to_dict() 
    df[ind] = df[ind].map(dics)
    df[ind].astype(int)

# Преобразуем переменне

In [54]:
df['Roof'] = df.RoofStyle * df.RoofMatl
df['Total_Bath'] = df.FullBath + df.HalfBath/2 + df.BsmtFullBath + df.BsmtHalfBath/2
df['TotalSF']=df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF'] 
df['Total_sqr_footage'] = df['BsmtFinSF1'] + df['BsmtFinSF2'] + df['1stFlrSF'] + df['2ndFlrSF']
df['Total_porch_sf'] = (df['OpenPorchSF'] + df['3SsnPorch'] + df['EnclosedPorch'] + df['ScreenPorch'] + df['WoodDeckSF'])
#df.drop(['RoofStyle','RoofMatl'],axis=1,inplace = True)
#df.drop(['FullBath','HalfBath','BsmtFullBath','BsmtHalfBath'],axis=1,inplace = True)
#df.drop(['TotalBsmtSF','1stFlrSF','2ndFlrSF'],axis=1,inplace = True)
#df.drop(['BsmtFinSF1','BsmtFinSF2'],axis=1,inplace = True)
#df.drop(['OpenPorchSF','3SsnPorch','EnclosedPorch','ScreenPorch','WoodDeckSF'],axis=1,inplace = True)

df = df.drop(['Utilities', 'Street', 'PoolQC',], axis=1)

#df['haspool'] = df['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
#df['has2ndfloor'] = df['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
#df['hasgarage'] = df['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
#df['hasbsmt'] = df['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
#df['hasfireplace'] = df['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)
#df = df.drop(['PoolArea', '2ndFlrSF', 'GarageArea','TotalBsmtSF','Fireplaces'], axis=1)

# Skewness

In [55]:
tt = df.drop(['indic','SalePrice'],axis=1,inplace = False)[df.drop(['indic','SalePrice'],axis=1,inplace = False).columns[abs(df.drop(['SalePrice'],inplace = False,axis = 1).skew())>1]].skew()
for t in tt.index:
    df[t] = boxcox1p(df[t],boxcox_normmax(df[t]+1))

# Целевая переменная

In [56]:
mask_SP = df.SalePrice.notna()
df.loc[mask_SP, ['SalePrice']] = np.log(df.loc[mask_SP, ['SalePrice']])
df[df.SalePrice.notna()].SalePrice = np.log(df[df.SalePrice.notna()].SalePrice)
df_train.SalePrice = np.log(df_train.SalePrice)
flag = False

# Метрика

In [57]:
def rmlse(y_true,y_pred):
    return np.sqrt(((np.log(y_true) - np.log(y_pred))**2).mean())
#RMLSE = make_scorer(root_mean_log_square_error)
#MSE = make_scorer(mean_squared_error)
#def neg_mse(y_true,y_pred):
#    return - mean_squared_error(y_true,y_pred)
#NEG_MSE = make_scorer(neg_mse)
def rmse(y_true,y_pred):
    return np.sqrt(((y_true - y_pred)**2).mean())

# Перейдем к фиттингу 

In [58]:
df_test, df_train = df[df["indic"].eq("test")], df[df["indic"].eq("train")]
X = df_train.drop(labels= ['indic','SalePrice'],axis=1,inplace = False)
Y = df_train.loc[:,'SalePrice'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
X_sub = df_test.drop(labels= ['indic','SalePrice'],axis=1,inplace = False)

# XGBoost

In [59]:
xg_simple = xgb.XGBRegressor().fit(X_train,y_train)
pred_xg_simple = xg_simple.predict(X_sub)
y_prediction_train = xg_simple.predict(X_train)
y_prediction_test = xg_simple.predict(X_test)
if flag:
    print('Score')
    print('Train:',xg_simple.score(X_train,y_train))
    print('Test: ',xg_simple.score(X_test,y_test))
    print('Answr:',xg_simple.score(X_sub,ans_arr))
    print('\n')
    print('RMLSE')
    print('Train:',rmlse(y_train,y_prediction_train))
    print('Test: ',rmlse(y_test,y_prediction_test))
    print('Answr:',rmlse(pred_xg_simple,ans_arr))
    submission = pd.DataFrame({'Id': Id_answr_Array, 'SalePrice': pred_xg_simple.reshape(1459,)})
    submission.to_csv("Submissions\submission_XGBoost_NonParameter.csv", index=False)
else:
    print('Score')
    print('Train:',xg_simple.score(X_train,y_train))
    print('Test: ',xg_simple.score(X_test,y_test))
    print('Answr:',xg_simple.score(X_sub,np.log(ans_arr)))
    print('\n')
    print('RMLSE')
    print('Train:',rmse(y_train,y_prediction_train))
    print('Test: ',rmse(y_test,y_prediction_test))
    print('Answr:',rmse(pred_xg_simple,np.log(ans_arr)))
    submission = pd.DataFrame({'Id': Id_answr_Array, 'SalePrice': np.exp(pred_xg_simple).reshape(1459,)})
    submission.to_csv("Submissions\submission_XGBoost_NonParameter.csv", index=False)

Score
Train: 0.999894542440993
Test:  0.8800769192321833
Answr: 0.8821688633168554


RMLSE
Train: 0.554673815035645
Test:  0.5675371225562056
Answr: 0.1429056371923106


In [60]:
xg_iparam = xgb.XGBRegressor(reg_lambda = 5.5, subsample=1, colsample_bytree = 0.3, learning_rate = 0.04,
                max_depth = 4, n_estimators = 729, alpha = 10).fit(X_train,y_train)
pred_xg_iparam = xg_iparam.predict(X_sub)
y_prediction_train = xg_iparam.predict(X_train)
y_prediction_test = xg_iparam.predict(X_test)
if flag:
    print('Score')
    print('Train:',xg_iparam.score(X_train,y_train))
    print('Test: ',xg_iparam.score(X_test,y_test))
    print('Answr:',xg_iparam.score(X_sub,ans_arr))
    print('\n')
    print('RMLSE')
    print('Train:',rmlse(y_train,y_prediction_train))
    print('Test: ',rmlse(y_test,y_prediction_test))
    print('Answr:',rmlse(pred_xg_iparam,ans_arr))
    submission = pd.DataFrame({'Id': Id_answr_Array, 'SalePrice': pred_xg_iparam.reshape(1459,)})
    submission.to_csv("Submissions\submission_XGBoost_Internet_Parameters.csv", index=False)
else: 
    print('Score')
    print('Train:',xg_iparam.score(X_train,y_train))
    print('Test: ',xg_iparam.score(X_test,y_test))
    print('Answr:',xg_iparam.score(X_sub,np.log(ans_arr)))
    print('\n')
    print('RMLSE')
    print('Train:',rmse(y_train,y_prediction_train))
    print('Test: ',rmse(y_test,y_prediction_test))
    print('Answr:',rmse(pred_xg_iparam,np.log(ans_arr)))
    submission = pd.DataFrame({'Id': Id_answr_Array, 'SalePrice': np.exp(pred_xg_iparam).reshape(1459,)})
    submission.to_csv("Submissions\submission_XGBoost_Internet_Parameters.csv", index=False)

Score
Train: 0.910092797553346
Test:  0.8645408529001163
Answr: 0.8695168432253199


RMLSE
Train: 0.530072464150975
Test:  0.5443894883395942
Answr: 0.15038224177095902


# Линейная Регрессия

In [61]:
lin_reg = LinearRegression().fit(X_train,y_train)
pred_lin_reg = lin_reg.predict(X_sub)
y_prediction_train = lin_reg.predict(X_train)
y_prediction_test = lin_reg.predict(X_test)
if (len(y_prediction_train[y_prediction_train<=0]) + len(y_prediction_test[y_prediction_test<=0]) + len(pred_lin_reg[pred_lin_reg<=0])) > 0:
    print('Есть отрицательные значения в предсказаниях!!!\n','Оценки RMLSE могуть быть неверными\n')
pred_lin_reg[pred_lin_reg<=0] = 1
y_prediction_train[y_prediction_train<=0] = 1
y_prediction_test[y_prediction_test<=0] = 1
if flag:
    print('Score')
    print('Train:',lin_reg.score(X_train,y_train))
    print('Test: ',lin_reg.score(X_test,y_test))
    print('Answr:',lin_reg.score(X_sub,ans_arr))
    print('\n')
    print('RMLSE')
    print('Train:',rmlse(y_train,y_prediction_train))
    print('Test: ',rmlse(y_test,y_prediction_test))
    print('Answr:',rmlse(pred_lin_reg,ans_arr))
    submission = pd.DataFrame({'Id': Id_answr_Array, 'SalePrice': pred_lin_reg.reshape(1459,)})
    submission.to_csv("Submissions\submission_LinearRegression_NonParameter.csv", index=False)
else:
    print('Score')
    print('Train:',lin_reg.score(X_train,y_train))
    print('Test: ',lin_reg.score(X_test,y_test))
    print('Answr:',lin_reg.score(X_sub,np.log(ans_arr)))
    print('\n')
    print('RMLSE')
    print('Train:',rmse(y_train,y_prediction_train))
    print('Test: ',rmse(y_test,y_prediction_test))
    print('Answr:',rmse(pred_lin_reg,np.log(ans_arr)))
    submission = pd.DataFrame({'Id': Id_answr_Array, 'SalePrice': np.exp(pred_lin_reg).reshape(1459,)})
    submission.to_csv("Submissions\submission_LinearRegression_NonParameter.csv", index=False)

Score
Train: 0.9161802113435342
Test:  0.8996611906288752
Answr: 0.9011910799863087


RMLSE
Train: 0.11360879886807261
Test:  0.13077034098482657
Answr: 0.5695015208069559


In [62]:
lasso =  Lasso(alpha = 1e-6,max_iter = 50000).fit(X_train,y_train)
pred_lasso = lasso.predict(X_sub)
y_prediction_train = lasso.predict(X_train)
y_prediction_test = lasso.predict(X_test)
print('Score')
print('Train:',lasso.score(X_train,y_train))
print('Test: ',lasso.score(X_test,y_test))
print('Answr:',lasso.score(X_sub,np.log(ans_arr)))
print('\n')
print('RMLSE')
print('Train:',rmse(y_train,y_prediction_train))
print('Test: ',rmse(y_test,y_prediction_test))
print('Answr:',rmse(pred_lasso,np.log(ans_arr)))

Score
Train: 0.9161745845490097
Test:  0.8995639935690684
Answr: 0.9012470013808684


RMLSE
Train: 0.5431838206578466
Test:  0.5632655887118329
Answr: 0.13082612349205622


# Черновик 

In [31]:
mse = make_scorer(mean_squared_error)
import warnings
warnings.filterwarnings('ignore')

In [37]:
parameters = {'alpha':np.linspace(0,0.001,1001)}
gcv = GridSearchCV(lasso, parameters)
gcv.fit(X, Y)
GridSearchCV(estimator=Lasso(max_iter=50000),param_grid=parameters,scoring = mse)
gcv.best_params_

{'alpha': 0.000125}

In [ ]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
params = {  
            'max_depth' :  [4,5,6],
            'n_estimators' :  np.linspace(1,1000,101).astype(int),
    'colsample_bytree' : np.linspace(0.01,1,51),
    'alpha'         : np.linspace(0,20,21),
    'learning_rate' : np.linspace(0.01,1,51),
    'reg_lambda' : np.linspace(0,20,21),
    #'subsample' : np.linspace(0,1,50)
        }
rcv = RandomizedSearchCV(xgb.XGBRegressor(),param_distributions=params,n_iter=500,scoring = mse,cv=2)
search = rcv.fit(X, Y)
#RandomizedSearchCV(estimator=xgb.XGBRegressor())
print(search.best_params_)
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time,'\n')

In [145]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)

grid_params = {'max_depth' :  [4,5,6],'n_estimators' : [700,750,800,850,900,950,1000],
              'colsample_bytree' : [0.05,0.075,0.1,0.125],
              'alpha': [5,10,15,20,25,30,35],
              'learning_rate' : [0.05,0.075,0.1,0.125],
              'reg_lambda' : [0,0.2,0.4,0.6,0.8]}
gcv = GridSearchCV(xgb.XGBRegressor(), grid_params)
gcv.fit(X, Y)
GridSearchCV(estimator=xgb.XGBRegressor(),param_grid=grid_params,scoring = mse,cv=2)
print(gcv.best_params_)
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time,'\n')

Start Time = 00:42:11
{'alpha': 0, 'colsample_bytree': 0.1, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 800, 'reg_lambda': 4}
End Time = 04:00:30 



In [65]:
xg_iparam = xgb.XGBRegressor(reg_lambda = 4, colsample_bytree = 0.1, learning_rate = 0.1,
                max_depth = 4, n_estimators = 800, alpha = 0).fit(X_train,y_train)
pred_xg_iparam = xg_iparam.predict(X_sub)
y_prediction_train = xg_iparam.predict(X_train)
y_prediction_test = xg_iparam.predict(X_test)
if flag:
    print('Score')
    print('Train:',xg_iparam.score(X_train,y_train))
    print('Test: ',xg_iparam.score(X_test,y_test))
    print('Answr:',xg_iparam.score(X_sub,ans_arr))
    print('\n')
    print('RMLSE')
    print('Train:',rmlse(y_train,y_prediction_train))
    print('Test: ',rmlse(y_test,y_prediction_test))
    print('Answr:',rmlse(pred_xg_iparam,ans_arr))
    submission = pd.DataFrame({'Id': Id_answr_Array, 'SalePrice': pred_xg_iparam.reshape(1459,)})
    submission.to_csv("Submissions\submission_XGBoost_Internet_Parameters.csv", index=False)
else: 
    print('Score')
    print('Train:',xg_iparam.score(X_train,y_train))
    print('Test: ',xg_iparam.score(X_test,y_test))
    print('Answr:',xg_iparam.score(X_sub,np.log(ans_arr)))
    print('\n')
    print('RMLSE')
    print('Train:',rmse(y_train,y_prediction_train))
    print('Test: ',rmse(y_test,y_prediction_test))
    print('Answr:',rmse(pred_xg_iparam,np.log(ans_arr)))
    submission = pd.DataFrame({'Id': Id_answr_Array, 'SalePrice': np.exp(pred_xg_iparam).reshape(1459,)})
    submission.to_csv("Submissions\submission_XGBoost_Internet_Parameters.csv", index=False)

Score
Train: 0.9983309469143217
Test:  0.9001483351530029
Answr: 0.9013363039769954


RMLSE
Train: 0.5532376662189767
Test:  0.5654079445352246
Answr: 0.13076695690929557


In [66]:
preds_blend = pred_xg_iparam*0.5+pred_lasso*0.5
print(rmse(preds_blend,np.log(ans_arr)))
submission = pd.DataFrame({'Id': Id_answr_Array, 'SalePrice': np.exp(preds_blend).reshape(1459,)})
submission.to_csv("Submissions\submission_Blender.csv", index=False)

0.12371085903264044


In [25]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,indic,SalePrice
0,6.502440,19.611113,13.318189,13.424521,11.459793,12.012112,0.000000,0.711828,41.795425,0.000000,0.0,16.0,4.901026,5.330072,142.010156,466.377437,7,5,2003,2003,0.541899,1.023583,11.0,12.0,2.0,19.378557,2.0,18.850213,5.0,3.0,11.627298,0.659828,6.0,117.625234,951.058101,0.0,150.0,294.084912,1.877499e+06,4.0,49.39966,259.35281,5.318378,854,0.0,7.234115,1.0,0.00000,2,1,3,0.75096,2.0,8,251740.88363,0,1.0,5.0,2003.0,2.0,2.0,548.0,11.872587,31.075909,10.4658,0.000000,12.009560,0.000000,0.0,0.0,0.0,0.0,8.425594,46.697731,0.0,2,2008,2.153401,85.375015,train,12.247694
1,4.250991,19.611113,14.895682,13.694528,11.459793,12.012112,0.000000,0.711828,41.795425,0.756051,0.0,20.0,2.276711,5.330072,142.010156,257.467291,6,8,1976,1976,0.541899,1.023583,4.0,5.0,1.0,0.000000,1.0,18.850213,2.0,3.0,11.627298,1.437294,4.0,146.501095,951.058101,0.0,284.0,403.274763,1.877499e+06,4.0,49.39966,259.35281,5.550626,0,0.0,6.947211,0.0,0.71081,2,0,3,0.75096,1.0,6,251740.88363,1,3.0,5.0,1976.0,2.0,2.0,460.0,11.872587,31.075909,10.4658,56.339208,0.000000,0.000000,0.0,0.0,0.0,0.0,8.425594,46.697731,0.0,5,2007,2.153401,85.375015,train,12.109011
2,6.502440,19.611113,13.647660,14.034134,11.459793,12.012112,0.670805,0.711828,41.795425,0.000000,0.0,16.0,4.901026,5.330072,142.010156,466.377437,7,5,2001,2002,0.541899,1.023583,11.0,12.0,2.0,17.721068,2.0,18.850213,5.0,3.0,11.627298,1.016484,6.0,91.408432,951.058101,0.0,434.0,311.854347,1.877499e+06,4.0,49.39966,259.35281,5.362015,866,0.0,7.275131,1.0,0.00000,2,1,3,0.75096,2.0,6,251740.88363,1,3.0,5.0,2001.0,2.0,2.0,608.0,11.872587,31.075909,10.4658,0.000000,9.849186,0.000000,0.0,0.0,0.0,0.0,8.425594,46.697731,0.0,9,2008,2.153401,85.375015,train,12.317167
3,6.865584,19.611113,12.751116,13.683422,11.459793,12.012112,0.670805,0.711828,41.795425,1.262204,0.0,17.0,4.901026,5.330072,142.010156,466.377437,7,5,1915,1970,0.541899,1.023583,5.0,8.0,1.0,0.000000,1.0,18.850213,1.0,2.0,20.326843,0.659828,4.0,52.695388,951.058101,0.0,540.0,265.805738,1.877499e+06,3.0,49.39966,259.35281,5.388291,756,0.0,7.237969,1.0,0.00000,1,0,3,0.75096,2.0,7,251740.88363,1,4.0,2.0,1998.0,1.0,3.0,642.0,11.872587,31.075909,10.4658,0.000000,8.921747,16.012024,0.0,0.0,0.0,0.0,8.425594,46.697731,0.0,2,2006,2.153401,3.021760,train,11.849398
4,6.502440,19.611113,15.289848,14.550143,11.459793,12.012112,0.670805,0.711828,41.795425,0.756051,0.0,24.0,4.901026,5.330072,142.010156,466.377437,8,5,2000,2000,0.541899,1.023583,11.0,12.0,2.0,25.323125,2.0,18.850213,5.0,3.0,11.627298,1.257256,6.0,111.824121,951.058101,0.0,490.0,372.598448,1.877499e+06,4.0,49.39966,259.35281,5.493031,1053,0.0,7.470740,1.0,0.00000,2,1,4,0.75096,2.0,9,251740.88363,1,3.0,5.0,2000.0,2.0,3.0,836.0,11.872587,31.075909,10.4658,42.351292,14.180239,0.000000,0.0,0.0,0.0,0.0,8.425594,46.697731,0.0,12,2008,2.153401,85.375015,train,12.429216


In [44]:
transformer = RobustScaler().fit_transform(df.LotArea.values.reshape(-1, 1))

In [54]:
np.sort((transformer - transformer.mean())/transformer.std())

array([[-0.12527156],
       [ 0.12706988],
       [ 0.44445737],
       ...,
       [ 1.63128131],
       [ 0.29460148],
       [ 0.13265411]])

In [70]:
(abs((df.LotArea - df.LotArea.mean())/df.LotArea.std())>7).sum()

1

In [82]:
for ind in df.drop('indic',axis=1).columns:
    number = (abs((df[ind] - df[ind].mean())/df[ind].std())>3).sum()
    if number > 0:
        print(ind,': ',number)

LotFrontage :  18
LotArea :  63
Street :  12
Alley :  198
LandContour :  237
Utilities :  1
LandSlope :  141
Condition1 :  151
Condition2 :  30
HouseStyle :  8
OverallQual :  4
OverallCond :  58
YearBuilt :  8
RoofStyle :  22
RoofMatl :  43
MasVnrArea :  14
ExterCond :  82
BsmtQual :  79
BsmtCond :  206
BsmtFinSF1 :  2
BsmtFinType2 :  52
BsmtFinSF2 :  114
BsmtUnfSF :  16
TotalBsmtSF :  13
Heating :  45
HeatingQC :  3
CentralAir :  196
Electrical :  59
1stFlrSF :  8
2ndFlrSF :  10
LowQualFinSF :  40
GrLivArea :  13
BsmtFullBath :  2
BsmtHalfBath :  175
FullBath :  4
HalfBath :  25
BedroomAbvGr :  30
KitchenAbvGr :  134
TotRmsAbvGrd :  19
Functional :  165
Fireplaces :  12
GarageYrBlt :  7
GarageCars :  1
GarageArea :  17
GarageQual :  190
GarageCond :  172
PavedDrive :  216
WoodDeckSF :  14
OpenPorchSF :  9
EnclosedPorch :  43
3SsnPorch :  37
ScreenPorch :  145
PoolArea :  13
PoolQC :  13
Fence :  12
MiscFeature :  105
MiscVal :  102
SaleType :  33
SalePrice :  12


In [26]:
lasso =  Lasso(alpha = 1e-6).fit(X_train,y_train)
pred_lasso = lasso.predict(X_sub)
y_prediction_train = lasso.predict(X_train)
y_prediction_test = lasso.predict(X_test)
print('Score')
print('Train:',lasso.score(X_train,y_train))
print('Test: ',lasso.score(X_test,y_test))
print('Answr:',lasso.score(X_sub,np.log(ans_arr)))
print('\n')
print('RMLSE')
print('Train:',rmse(y_train,y_prediction_train))
print('Test: ',rmse(y_test,y_prediction_test))
print('Answr:',rmse(pred_lasso,np.log(ans_arr)))

Score
Train: 0.9128310599922996
Test:  0.8951044000473967
Answr: 0.896490905882851


RMLSE
Train: 0.542718509307058
Test:  0.5622201353423856
Answr: 0.13393947151411087


In [25]:
lasso =  Lasso(alpha = 5e-06,max_iter = 100000).fit(X_train,y_train)
pred_lasso = lasso.predict(X_sub)
y_prediction_train = lasso.predict(X_train)
y_prediction_test = lasso.predict(X_test)
print('Score')
print('Train:',lasso.score(X_train,y_train))
print('Test: ',lasso.score(X_test,y_test))
print('Answr:',lasso.score(X_sub,np.log(ans_arr)))
print('\n')
print('RMLSE')
print('Train:',rmse(y_train,y_prediction_train))
print('Test: ',rmse(y_test,y_prediction_test))
print('Answr:',rmse(pred_lasso,np.log(ans_arr)))

Score
Train: 0.921382250991374
Test:  0.8982029999962489
Answr: 0.7980180565099715


RMLSE
Train: 0.5438712244284554
Test:  0.5641117161434824
Answr: 0.18710078105400063


In [64]:
lasso =  Lasso(alpha = 1e-8,max_iter = 100000).fit(X_train,y_train)
pred_lasso = lasso.predict(X_sub)
y_prediction_train = lasso.predict(X_train)
y_prediction_test = lasso.predict(X_test)
print('Score')
print('Train:',lasso.score(X_train,y_train))
print('Test: ',lasso.score(X_test,y_test))
print('Answr:',lasso.score(X_sub,np.log(ans_arr)))
print('\n')
print('RMLSE')
print('Train:',rmse(y_train,y_prediction_train))
print('Test: ',rmse(y_test,y_prediction_test))
print('Answr:',rmse(pred_lasso,np.log(ans_arr)))

Score
Train: 0.9161802107392483
Test:  0.8996603053738416
Answr: 0.9011917469423032


RMLSE
Train: 0.5431962403203617
Test:  0.5632618645512684
Answr: 0.13086271839666774


In [63]:
lasso =  Lasso(alpha = 0.000125,max_iter = 100000).fit(X_train,y_train)
pred_lasso = lasso.predict(X_sub)
y_prediction_train = lasso.predict(X_train)
y_prediction_test = lasso.predict(X_test)
print('Score')
print('Train:',lasso.score(X_train,y_train))
print('Test: ',lasso.score(X_test,y_test))
print('Answr:',lasso.score(X_sub,np.log(ans_arr)))
print('\n')
print('RMLSE')
print('Train:',rmse(y_train,y_prediction_train))
print('Test: ',rmse(y_test,y_prediction_test))
print('Answr:',rmse(pred_lasso,np.log(ans_arr)))

Score
Train: 0.9154161785559704
Test:  0.8988748870652737
Answr: 0.9015670040182989


RMLSE
Train: 0.5428099717680699
Test:  0.5628245497349227
Answr: 0.13061398474604857
